In [21]:
'''
1. GHS for repos x
2. PyDriller or SrcML x
3. Preprocessing x
4. Tokenization x
5. Model
6. Evaluation (perplexity)
7. Training data -> pick the best model
8. Test -> Perplexity
9. Writeup (1 page)
'''

'\n1. GHS for repos x\n2. PyDriller or SrcML x\n3. Preprocessing x\n4. Tokenization x\n5. Model\n6. Evaluation (perplexity)\n7. Training data -> pick the best model\n8. Test -> Perplexity\n9. Writeup (1 page)\n'

In [22]:
!pip install pydriller
!pip install javalang

In [23]:
!add-apt-repository ppa:git-core/ppa
!apt-get update
!apt-get install git


PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/git-core/ppa/ubuntu/ jammy main'
Description:
The most current stable version of Git for Ubuntu.

For release candidates, go to https://launchpad.net/~git-core/+archive/candidate .
More info: https://launchpad.net/~git-core/+archive/ubuntu/ppa
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Found existing deb entry in /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding deb entry to /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Found existing deb-src entry in /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding key to /etc/apt/trusted.gpg.d/git-core-ubuntu-ppa.gpg with fingerprint F911AB184317630C59970973E363C90F8F1B6217
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-proje

In [24]:
!add-apt-repository ppa:git-core/ppa
!apt-get update
!apt-get install git



PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/git-core/ppa/ubuntu/ jammy main'
Description:
The most current stable version of Git for Ubuntu.

For release candidates, go to https://launchpad.net/~git-core/+archive/candidate .
More info: https://launchpad.net/~git-core/+archive/ubuntu/ppa
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Found existing deb entry in /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding deb entry to /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Found existing deb-src entry in /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/git-core-ubuntu-ppa-jammy.list
Adding key to /etc/apt/trusted.gpg.d/git-core-ubuntu-ppa.gpg with fingerprint F911AB184317630C59970973E363C90F8F1B6217
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/

In [36]:

import pandas as pd # For both
from pydriller import Repository
import os
import re # For preprocessing
from javalang.parse import parse
from javalang.tree import MethodDeclaration

In [37]:
df_res = pd.read_csv('results.csv')

In [38]:
repoList = []
for idx,row in df_res.iterrows():
  repoList.append("https://www.github.com/{}".format(row['name']))
repoList[0:10]

['https://www.github.com/dustin/java-memcached-client',
 'https://www.github.com/davidb/scala-maven-plugin',
 'https://www.github.com/tcurdt/jdeb',
 'https://www.github.com/rictic/code_swarm',
 'https://www.github.com/mth/yeti',
 'https://www.github.com/myabc/nbgit',
 'https://www.github.com/cyberfox/jbidwatcher',
 'https://www.github.com/mheath/adbcj',
 'https://www.github.com/rikrd/geomerative',
 'https://www.github.com/raykrueger/hibernate-memcached']

In [43]:
from pydriller import Repository
import os
import csv
from javalang.parse import parse
from javalang.tree import MethodDeclaration
import javalang

def extract_methods_from_java(code):
    """
    Extract methods from Java source code using javalang parser.

    Args:
        code (str): The Java source code.

    Returns:
        list: A list of tuples containing method names and their full source code.
    """
    methods = []
    try:
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = javalang.parse.parse(code)
        lines = code.splitlines()

        # Traverse the tree to find method declarations
        for _, node in tree.filter(javalang.tree.MethodDeclaration):
            method_name = node.name

            # Determine the start and end lines of the method
            start_line = node.position.line - 1
            end_line = None

            # Use the body of the method to determine its end position
            if node.body:
                last_statement = node.body[-1]
                if hasattr(last_statement, 'position') and last_statement.position:
                    end_line = last_statement.position.line

            # Extract method code
            if end_line:
                method_code = "\n".join(lines[start_line:end_line+1])
            else:
                # If end_line couldn't be determined, extract up to the end of the file
                method_code = "\n".join(lines[start_line:])

            methods.append((method_name, method_code))

    except Exception as e:
        print(f"Error parsing Java code: {e}")
    return methods


def extract_methods_to_csv_from_master(repo_path, output_csv):
    """
    Extract methods from Java files in the master branch and append them to a CSV file.

    Args:
        repo_path (str): Path to the Git repository.
        output_csv (str): Path to the output CSV file.
    """
    method_count = 0  # Counter for methods extracted
    # Check if the CSV file exists already
    file_exists = os.path.exists(output_csv)
    with open(output_csv, mode='a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write header only if file does not exist
        if not file_exists:
            csv_writer.writerow(["Commit Hash", "File Name", "Method Name", "Method Code", "Commit Link"])

        for commit in Repository(repo_path, only_in_branch="master").traverse_commits():
            print(f"Processing commit: {commit.hash}")
            for modified_file in commit.modified_files:
                if modified_file.filename.endswith(".java") and modified_file.source_code:
                    methods = extract_methods_from_java(modified_file.source_code)
                    for method_name, method_code in methods:
                        commit_link = f"{repo_path}/commit/{commit.hash}"
                        csv_writer.writerow([commit.hash, modified_file.filename, method_name, method_code, commit_link])
                        method_count += 1
                    print(f"Extracted {len(methods)} methods from {modified_file.filename} in commit {commit.hash}")
                    print(f"Total methods extracted so far: {method_count}")
    print(f"Finished extracting methods. Total methods extracted: {method_count}")


def extract_methods_to_csv(repo_path, output_csv):
    """
    Extract methods from Java files in a repository and append them to a CSV file.

    Args:
        repo_path (str): Path to the Git repository.
        output_csv (str): Path to the output CSV file.
    """
    method_count = 0  # Counter for methods extracted
    file_exists = os.path.exists(output_csv)
    with open(output_csv, mode='a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        if not file_exists:
            csv_writer.writerow(["Branch Name", "Commit Hash", "File Name", "Method Name", "Method Code", "Commit Link"])
        branch_name = "master"
        for commit in Repository(repo_path, only_in_branch=branch_name).traverse_commits():
            print(f"Processing commit: {commit.hash}")
            for modified_file in commit.modified_files:
                if modified_file.filename.endswith(".java") and modified_file.source_code:
                    methods = extract_methods_from_java(modified_file.source_code)
                    for method_name, method_code in methods:
                        commit_link = f"{repo_path}/commit/{commit.hash}"
                        csv_writer.writerow([branch_name, commit.hash, modified_file.filename, method_name, method_code, commit_link])
                        method_count += 1
                    print(f"Extracted {len(methods)} methods from {modified_file.filename} in commit {commit.hash}")
                    print(f"Total methods extracted so far: {method_count}")
    print(f"Finished extracting methods. Total methods extracted: {method_count}")


# Example usage:
for repo in repoList[0:20]:
    fileNameToSave = ''.join(repo.split('github.com')[1:])
    fileNameToSave = fileNameToSave.replace('/', '_')
    # Specify the path to the output CSV file
    output_csv_file = "extracted_methods.csv"
    # Run the extraction
    extract_methods_to_csv_from_master(repo, output_csv_file)
    print(repo)




Streaming output truncated to the last 5000 lines.
Total methods extracted so far: 73799
Processing commit: 4b7643e856c861c18fe646647cfba05b45c4e6e2
Extracted 1 methods from DomainObjectTest.java in commit 4b7643e856c861c18fe646647cfba05b45c4e6e2
Total methods extracted so far: 73800
Extracted 8 methods from GenerateDomainCodegenPass.java in commit 4b7643e856c861c18fe646647cfba05b45c4e6e2
Total methods extracted so far: 73808
Processing commit: 31c219719119cd26fc838ffe3482c030c0555a03
Extracted 41 methods from ChildCodegen.java in commit 31c219719119cd26fc838ffe3482c030c0555a03
Total methods extracted so far: 73849
Extracted 46 methods from CodeADomainObjectCodegen.java in commit 31c219719119cd26fc838ffe3482c030c0555a03
Total methods extracted so far: 73895
Extracted 32 methods from GrandChildCodegen.java in commit 31c219719119cd26fc838ffe3482c030c0555a03
Total methods extracted so far: 73927
Extracted 80 methods from HistoryEntryCodegen.java in commit 31c219719119cd26fc838ffe3482c030c

In [44]:
### Type 1 Clones ###
def remove_duplicates(data):
    """Remove duplicate methods based on method content.
      Almost Type-1 with the exception of comments
    """
    return data.drop_duplicates(subset="Method Java", keep="first")

def filter_ascii_methods(data):
    """Filter methods to include only those with ASCII characters."""
    data = data[data["Method Java"].apply(lambda x: all(ord(char) < 128 for char in x))]
    return data

# Three Approaches:
# 	1.	Data Distribution-Based Filtering: We eliminate outliers by analyzing the original data distribution, as demonstrated below.
# 	2.	Literature-Driven Filtering: We follow best practices outlined in research, such as removing methods exceeding 512 tokens in length.
# 	3.	Hybrid Approach: We combine elements from both the distribution-based and literature-driven methods.

def remove_outliers(data, lower_percentile=5, upper_percentile=95):
    """Remove outliers based on method length."""
    method_lengths = data["Method Java"].apply(len)
    lower_bound = method_lengths.quantile(lower_percentile / 100)
    upper_bound = method_lengths.quantile(upper_percentile / 100)
    return data[(method_lengths >= lower_bound) & (method_lengths <= upper_bound)]

## Tokenization

In [45]:
import pandas as pd
from pygments.lexers import get_lexer_by_name
from pygments.token import Token
import re
from IPython.display import display

# Load methods from CSV
data = pd.read_csv('extracted_methods.csv')

# ---------------------- Preprocessing ----------------------
def remove_duplicates(data):
    return data.drop_duplicates(subset="Method Code", keep="first")

def filter_ascii_methods(data):
    return data[data["Method Code"].apply(lambda x: all(ord(char) < 128 for char in str(x)))]

def remove_outliers(data, lower_percentile=5, upper_percentile=95):
    method_lengths = data["Method Code"].apply(len)
    lower_bound = method_lengths.quantile(lower_percentile / 100)
    upper_bound = method_lengths.quantile(upper_percentile / 100)
    return data[(method_lengths >= lower_bound) & (method_lengths <= upper_bound)]

def remove_boilerplate_methods(data):
    boilerplate_patterns = [
        r"\bset[A-Z][a-zA-Z0-9_]*\(.*\)\s*{",  # Setter methods
        r"\bget[A-Z][a-zA-Z0-9_]*\(.*\)\s*{",  # Getter methods
    ]
    boilerplate_regex = re.compile("|".join(boilerplate_patterns))
    return data[~data["Method Code"].apply(lambda x: bool(boilerplate_regex.search(str(x))))]

def remove_comments_from_dataframe(df, method_column, language):
    def remove_comments(code):
        lexer = get_lexer_by_name(language)
        tokens = lexer.get_tokens(code)
        return ''.join(token[1] for token in tokens if not (lambda t: t[0] in Token.Comment)(token))

    df["Method Code No Comments"] = df[method_column].apply(remove_comments)
    return df


# ---------------------- Tokenization with Cleaning ----------------------
def clean_and_tokenize(text):
    """Tokenize the text while removing tabs, newlines, and extra spaces."""
    # Remove leading/trailing whitespace and tabs
    text = text.strip()

    # Remove unwanted characters like \t, \n, and multiple spaces
    text = re.sub(r'[\t\n\r]+', ' ', text)  # Replace tabs/newlines with space
    text = re.sub(r'\s+', ' ', text)        # Replace multiple spaces with single space

    # Tokenize using Pygments (lexer-based)
    lexer = get_lexer_by_name("java")
    tokens = [t[1].strip() for t in lexer.get_tokens(text) if t[1].strip() != '']
    return tokens


def tokenize_methods(df):
    df["Tokens"] = df["Method Code No Comments"].apply(clean_and_tokenize)
    df["Token Count"] = df["Tokens"].apply(len)
    return df


# ---------------------- Apply Processing Steps ----------------------
data = remove_duplicates(data)
data = filter_ascii_methods(data)
data = remove_outliers(data)
data = remove_boilerplate_methods(data)
data = remove_comments_from_dataframe(data, "Method Code", "java")
processed_data = tokenize_methods(data)


# ---------------------- Display Result ----------------------
display(processed_data[['Method Code No Comments', 'Tokens', 'Token Count']].head())


,Method Code No Comments,Tokens,Token Count
0,\tpublic void storeAsync(StoreOperation.StoreT...,"[public, void, storeAsync, (, StoreOperation, ...",51
1,\tpublic String storeSync(StoreOperation.Store...,"[public, String, storeSync, (, StoreOperation,...",91
2,\t\t\t\t\tpublic void storeResult(String val) ...,"[public, void, storeResult, (, String, val, ),...",16
3,"\tpublic String add(String key, int flags, int...","[public, String, add, (, String, key, ,, int, ...",38
4,"\tpublic String set(String key, int flags, int...","[public, String, set, (, String, key, ,, int, ...",38


In [46]:
processed_data
print(processed_data["Token Count"].describe())


count    80635.000000
mean       123.124363
std        220.415155
min          6.000000
25%         28.000000
50%         54.000000
75%        120.000000
max       5340.000000
Name: Token Count, dtype: float64


In [47]:
import collections
import math
import random
import json

# ----------------- N-gram Model Functions -----------------

def train_ngram_model(corpus, n):
    """
    Trains an n-gram model on the given corpus.

    Args:
        corpus (list of list of str): A list where each element is a list of tokens.
        n (int): The n value for the n-gram.

    Returns:
        ngram_counts: Counter mapping n-gram tuples to counts.
        context_counts: Counter mapping (n-1)-gram tuples (contexts) to counts.
        vocabulary: Set of tokens in the corpus.
    """
    ngram_counts = collections.Counter()
    context_counts = collections.Counter()
    vocabulary = set()

    for tokens in corpus:
        vocabulary.update(tokens)
        # Pad with (n-1) start tokens and an end token
        padded_tokens = ['<s>'] * (n - 1) + tokens + ['</s>']
        for i in range(len(padded_tokens) - n + 1):
            ngram = tuple(padded_tokens[i:i+n])
            context = ngram[:-1]
            ngram_counts[ngram] += 1
            context_counts[context] += 1

    return ngram_counts, context_counts, vocabulary

def ngram_probability(ngram, ngram_counts, context_counts, vocab_size, smoothing=1):
    """
    Computes the probability of an n-gram using Laplace smoothing.
    """
    context = ngram[:-1]
    count_ngram = ngram_counts.get(ngram, 0)
    count_context = context_counts.get(context, 0)
    return (count_ngram + smoothing) / (count_context + smoothing * vocab_size)

def compute_perplexity(corpus, n, ngram_counts, context_counts, vocab_size, smoothing=1):
    """
    Computes perplexity of an n-gram model on a corpus.
    """
    log_prob_sum = 0.0
    M = 0  # Total number of n-gram tokens (excluding padding tokens)

    for tokens in corpus:
        padded_tokens = ['<s>'] * (n - 1) + tokens + ['</s>']
        for i in range(len(padded_tokens) - n + 1):
            ngram = tuple(padded_tokens[i:i+n])
            prob = ngram_probability(ngram, ngram_counts, context_counts, vocab_size, smoothing)
            log_prob_sum += math.log(prob)
            M += 1
    return math.exp(-log_prob_sum / M) if M > 0 else float('inf')

def generate_completion(initial_tokens, n, ngram_counts, context_counts, vocabulary, max_tokens=20, smoothing=1):
    """
    Generates a token sequence by predicting the next token repeatedly,
    and records the step-by-step prediction process.
    """
    generated = initial_tokens.copy()
    steps = []  # Records each prediction step.

    for _ in range(max_tokens):
        # Use last (n-1) tokens as context; pad if needed.
        context = tuple((['<s>'] * max(0, n - 1 - len(generated)) + generated)[- (n - 1):])
        candidates = {}
        for token in vocabulary.union({'</s>'}):
            ngram = context + (token,)
            prob = ngram_probability(ngram, ngram_counts, context_counts, len(vocabulary), smoothing)
            if prob > 0:
                candidates[token] = prob
        if not candidates:
            break
        total_prob = sum(candidates.values())
        tokens, probs = zip(*[(t, p / total_prob) for t, p in candidates.items()])
        next_token = random.choices(tokens, weights=probs)[0]

        steps.append({
            "context": list(context),
            "predicted_token": next_token,
            "predicted_probability": candidates[next_token] / total_prob
        })

        if next_token == '</s>':
            break
        generated.append(next_token)
    return generated, steps

# ----------------- Prepare Corpus -----------------

# Assuming 'processed_data' DataFrame exists and has a 'Tokens' column.
all_methods = processed_data['Tokens'].tolist()

# Shuffle and split into train (80%), eval (10%), test (10%)
random.seed(42)
random.shuffle(all_methods)
n_total = len(all_methods)
n_train = int(0.8 * n_total)
n_eval = int(0.1 * n_total)
train_corpus = all_methods[:n_train]
eval_corpus = all_methods[n_train:n_train+n_eval]
test_corpus = all_methods[n_train+n_eval:]

print("Corpus sizes: Train = {}, Eval = {}, Test = {}".format(len(train_corpus), len(eval_corpus), len(test_corpus)))

# ----------------- Model Training, Evaluation, and Selection -----------------

ngram_values = [3, 4, 5, 6, 7, 8, 9, 10]
results = {}  # Store perplexity scores for each n
best_n = None
best_perplexity = float('inf')
best_model = None

for n in ngram_values:
    print(f"\nTraining {n}-gram model...")
    ngram_counts, context_counts, vocabulary = train_ngram_model(train_corpus, n)
    perplexity = compute_perplexity(eval_corpus, n, ngram_counts, context_counts, len(vocabulary))
    results[n] = perplexity
    print(f"n = {n} -> Evaluation Perplexity: {perplexity:.2f}")
    if perplexity < best_perplexity:
        best_perplexity = perplexity
        best_n = n
        best_model = (ngram_counts, context_counts, vocabulary)

print(f"\nBest model: {best_n}-gram with perplexity {best_perplexity:.2f}")

# ----------------- Code Completion on Test Set with Ground Truth -----------------

completions = []
detailed_steps = {}  # Step-by-step predictions for each test instance

ngram_counts, context_counts, vocabulary = best_model

# For each test method, use the first (n-1) tokens as initial context and record the ground truth.
for i, tokens in enumerate(test_corpus[:100]):
    initial_context = tokens[:best_n-1] if len(tokens) >= best_n - 1 else tokens
    ground_truth = tokens[len(initial_context):]  # Actual continuation after initial context

    generated, steps = generate_completion(initial_context, best_n, ngram_counts, context_counts, vocabulary)

    # Compute probability for last predicted n-gram (if available)
    if len(generated) >= best_n:
        predicted_ngram = tuple(generated[-best_n:])
        pred_prob = ngram_probability(predicted_ngram, ngram_counts, context_counts, len(vocabulary))
    else:
        predicted_ngram = None
        pred_prob = None

    completions.append({
        "test_index": i,
        "initial_context": initial_context,
        "ground_truth_continuation": ground_truth,
        "predicted_completion": generated[len(initial_context):],
        "predicted_ngram": predicted_ngram,
        "predicted_probability": pred_prob
    })
    detailed_steps[f"test_index_{i}"] = steps

# Save results to JSON files
results_dict = {
    "model": {
        "n": best_n,
        "evaluation_perplexity": best_perplexity,
        "all_perplexities": results
    },
    "completions": completions
}

with open("results_student_model.json", "w", encoding="utf-8") as f:
    json.dump(results_dict, f, indent=4)

with open("detailed_steps.json", "w", encoding="utf-8") as f:
    json.dump(detailed_steps, f, indent=4)

# ----------------- Print Ground Truth vs. Predicted Completions (First 5 Test Instances) -----------------

print("\nGround Truth vs. Predicted Completions (First 5 Test Instances):")
for comp in completions[:5]:
    print(f"\nTest Index: {comp['test_index']}")
    print("Initial Context:")
    print(" ".join(comp["initial_context"]))
    print("Ground Truth Continuation:")
    print(" ".join(comp["ground_truth_continuation"]))
    print("Predicted Completion:")
    print(" ".join(comp["predicted_completion"]))

# Optionally, print a few step-by-step predictions for the first test instance.
print("\nStep-by-step predictions for the first test instance:")
for step in detailed_steps.get("test_index_0", []):
    print(f"Context: {step['context']} -> Predicted: {step['predicted_token']} (Prob: {step['predicted_probability']:.4f})")

print("\nResults saved to 'results_student_model.json' and 'detailed_steps.json'")


Corpus sizes: Train = 64508, Eval = 8063, Test = 8064

Training 3-gram model...
n = 3 -> Evaluation Perplexity: 303.87

Training 4-gram model...
n = 4 -> Evaluation Perplexity: 747.13

Training 5-gram model...
n = 5 -> Evaluation Perplexity: 1411.98

Training 6-gram model...
n = 6 -> Evaluation Perplexity: 2131.00

Training 7-gram model...
n = 7 -> Evaluation Perplexity: 2743.02

Training 8-gram model...
n = 8 -> Evaluation Perplexity: 3240.93

Training 9-gram model...
n = 9 -> Evaluation Perplexity: 3658.68

Training 10-gram model...
n = 10 -> Evaluation Perplexity: 4009.01

Best model: 3-gram with perplexity 303.87

Ground Truth vs. Predicted Completions (First 5 Test Instances):

Test Index: 0
Initial Context:
public void
Ground Truth Continuation:
fatal ( Object message ) { if ( realLogger ! = null ) realLogger . log ( Level . SEVERE , message . toString ( ) ) ;
Predicted Completion:
setUp itsStackDepth PartitionKeyStatistics forced getPositionForPopup ReaderInputStream View filese